In [58]:
import pandas as pd
import copy
import numpy as np
from collections import Counter

# TODOs

## Partner

- <span style="color:red">BUG: more wins than games for (probably) games with 3rd player</span>
- <span style="color:orange">IMPORTANT: also NEMESIS -> already implemented -> see R</span>
- specify when 3 ppl played together
-> re-structure probably: build df iteratively 

## Misc

In [59]:
spiele = pd.read_excel('Spiele.xlsx')
plätze = pd.read_excel('Platzierungen.xlsx')

spiele.fillna(value="", inplace = True)
plätze.fillna(value="", inplace = True)

# rename columns
spiele = spiele.rename(columns = {'Player 1': 'p1', 'Player 2': 'p2', 'Player 3': 'p3', 'Player 4': 'p4', 
                                  'Player 5': 'p5', 'Player 6': 'p6', 'Teamname 1': 'tname1', 'Teamname 2': 'tname2',
                                 'Datum': 'datum', 'W/L Team 1': 'WLt1', 'W/L Team 2': 'WLt2', 'Becher übrig': 'becher',
                                 'Trickshot': 'trickshot', 'Fenster Team': 'fenster', 'Verlängerung': 'verl',
                                 'Becher P1': 'p1Becher', 'Becher P2': 'p2Becher', 'Becher P3': 'p3Becher', 
                                 'Becher T1': 't1Becher', 'Becher P4': 'p4Becher', 'Becher P5': 'p5Becher',
                                  'Becher P6': 'p6Becher', 'Becher T2': 't2Becher', '1v1': '1v1', 
                                  'X_Runden Regel': '5called', 'Eigentore': 'Eigentore'})

# Apply str.strip() to remove trailing spaces from each cell
spiele = spiele.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# define player columns
player_cols  = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

# define all player(names)
players = pd.unique(spiele[player_cols].values.flatten())
players = players.tolist()
# remove empty name
players = [value for value in players if value]
# sort alphabetically
players.sort()

In [110]:
# make sure either both teams lost (X Runden gecalled) or exactly one team lost and excatly one team won
rule = spiele.apply(lambda row: (row['WLt1'], row['WLt2']) in [(0, 1), (1, 0), (0, 0)], axis=1)

if rule.all():
    print("Fine")
else:
    print('Fehler')


Fine


In [60]:
players

['Aaron',
 'Alina',
 'André',
 'Basti',
 'Chris',
 'Cuba',
 'Daisy',
 'Eggi',
 'Emilio',
 'Friedrich',
 'Gina',
 'Hannes',
 'Henny',
 'Jonte',
 'Konni',
 'Konrad',
 'Leon',
 'Lin',
 'Lorenz',
 'Luca',
 'Marian',
 'Marie',
 'Max',
 'Merlin',
 'Michi',
 'Moritz',
 'Mümmel',
 'Nikita',
 'Nora',
 'Ole',
 'Olek',
 'Phia',
 'Philipp',
 'Richard',
 'Tabea',
 'Timo',
 'Tobi',
 'Valdemar',
 'Verena',
 'Vincent',
 'Zoë']

In [61]:
spiele

,p1,p2,p3,p4,p5,p6,tname1,tname2,datum,WLt1,...,p2Becher,p3Becher,t1Becher,p4Becher,p5Becher,p6Becher,t2Becher,1v1,X-Runden Regel,Eigentore
0,Basti,Valdemar,,Emilio,Philipp,,Einmal mit und einmal ohne Socken,Saufomat,2021-11-26,0,...,,,,,,,,,,
1,Phia,Friedrich,,André,Timo,,Team Pferd *hüüh*,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
2,Phia,Friedrich,,Basti,Valdemar,,Team Pferd *hüüh*,Einmal mit und einmal ohne Socken,2021-11-26,1,...,,,,,,,,,,
3,Emilio,Philipp,,André,Timo,,Saufomat,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
4,Emilio,Philipp,,Phia,Friedrich,,Saufomat,Team Pferd *hüüh*,2021-11-26,0,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,Phia,André,,Philipp,Valdemar,,Pieps ÖHH & KraKraKra,Borosboiiis,2024-05-26,1,...,3.0,,6.0,2.0,2.0,,4.0,,,
228,Phia,Chris,,André,Marian,,DD-Detlef und Doris,Geckabor Gangster,2024-05-26,0,...,2.0,,3.0,3.0,3.0,,6.0,,,
229,Emilio,Valdemar,,Phia,Chris,,FDP Ferbrecher,DD-Detlef und Doris,2024-05-26,1,...,1.0,,6.0,2.0,0.0,,2.0,,,
230,André,Marian,,Emilio,Valdemar,,Geckabor Gangster,FDP Ferbrecher,2024-05-26,0,...,4.0,,6.0,4.0,5.0,,9.0,,,


In [62]:
plätze

,Player 1,Player 2,Player 3,Teamname,Datum,Platzierung,Anzahl Teams,Draft,Datum unsicher
0,Jonte,Valdemar,,Schöne & Biest,2021-08-07,1,4,0,
1,Michi,André,,Valdemars Albtraum,2021-08-07,2,4,0,
2,Daisy,Marie,,Immo-rich,2021-08-07,4,4,0,
3,Phia,Emilio,,Support Main,2021-08-07,2,4,0,
4,Basti,Valdemar,,Einmal mit und einmal ohne Socken,2021-11-26,3,4,1,
...,...,...,...,...,...,...,...,...,...
148,Phia,André,,Pieps ÖHH & KraKraKra,2024-05-26,1,4,1,
149,Marian,Chris,,TT-Torsten und Thorben,2024-05-26,3,4,1,
150,Phia,Chris,,DD-Detlef und Doris,2024-05-26,3,3,1,
151,Emilio,Valdemar,,FDP Ferbrecher,2024-05-26,1,3,1,


In [101]:
def winrate(player):
    df = spiele.copy(deep=True)
    pattern = rf'\b{player}\b'
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    
    team = -1
    games = 0
    wins = 0
    partner_dict = {elem: [0, 0] for elem in players}
    player_number = 'p0'
    
    for i in range(len(df)):
        partner1, partner2 = '', ''
        p1
        
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            partner1, partner2 = df.loc[i, 'p2'], df.loc[i, 'p3']
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            partner1, partner2 = df.loc[i, 'p1'], df.loc[i, 'p3']
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            partner1, partner2 = df.loc[i, 'p1'], df.loc[i, 'p2']
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            partner1, partner2 = df.loc[i, 'p5'], df.loc[i, 'p6']
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            partner1, partner2 = df.loc[i, 'p4'], df.loc[i, 'p6']
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            partner1, partner2 = df.loc[i, 'p4'], df.loc[i, 'p5']
        else:
            print(f"Player {player} not found in row {i}")
            continue
        
        if partner1:
            partner_dict[partner1][0] += 1
        if partner2:
            partner_dict[partner2][0] += 1
        
        games += 1
        if team == 1:
            current_win = df.loc[i, 'WLt1']
        else:
            current_win = df.loc[i, 'WLt2']
        
        if current_win:
            wins += 1
            if partner1:
                partner_dict[partner1][1] += 1
            if partner2:
                partner_dict[partner2][1] += 1
    
    winrate = wins / games * 100
    print(f'Total games: {games}')
    print(f'Winrate {player}: {winrate:.2f}%\n')
    
    for elem in partner_dict:
        games_tmp = partner_dict[elem][0]
        wins_tmp = partner_dict[elem][1]
        if games_tmp != 0:
            winrate_tmp = wins_tmp / games_tmp
        else:
            winrate_tmp = 0
        partner_dict[elem].append(winrate_tmp)
    
    return partner_dict

In [103]:
andre_partner = winrate('André')

Total games: 118
Winrate André: 53.39%



In [104]:
andre_partner

{'Aaron': [6, 2, 0.3333333333333333],
 'Alina': [0, 0, 0],
 'André': [0, 0, 0],
 'Basti': [3, 1, 0.3333333333333333],
 'Chris': [20, 11, 0.55],
 'Cuba': [0, 0, 0],
 'Daisy': [7, 0, 0.0],
 'Eggi': [0, 0, 0],
 'Emilio': [16, 10, 0.625],
 'Friedrich': [0, 0, 0],
 'Gina': [2, 0, 0.0],
 'Hannes': [0, 0, 0],
 'Henny': [3, 3, 1.0],
 'Jonte': [0, 0, 0],
 'Konni': [0, 0, 0],
 'Konrad': [0, 0, 0],
 'Leon': [0, 0, 0],
 'Lin': [0, 0, 0],
 'Lorenz': [0, 0, 0],
 'Luca': [9, 2, 0.2222222222222222],
 'Marian': [15, 9, 0.6],
 'Marie': [3, 1, 0.3333333333333333],
 'Max': [0, 0, 0],
 'Merlin': [7, 2, 0.2857142857142857],
 'Michi': [4, 1, 0.25],
 'Moritz': [0, 0, 0],
 'Mümmel': [5, 5, 1.0],
 'Nikita': [0, 0, 0],
 'Nora': [0, 0, 0],
 'Ole': [0, 0, 0],
 'Olek': [0, 0, 0],
 'Phia': [24, 17, 0.7083333333333334],
 'Philipp': [4, 4, 1.0],
 'Richard': [0, 0, 0],
 'Tabea': [0, 0, 0],
 'Timo': [2, 0, 0.0],
 'Tobi': [0, 0, 0],
 'Valdemar': [16, 12, 0.75],
 'Verena': [0, 0, 0],
 'Vincent': [0, 0, 0],
 'Zoë': [0, 0, 

In [94]:
spiele['p6'].value_counts()

p6
            172
Daisy         9
Valdemar      7
Marian        5
Mümmel        4
Phia          4
Max           3
Emilio        3
Marie         3
Merlin        3
Lorenz        3
Cuba          2
Konni         2
Nora          2
André         2
Chris         2
Ole           2
Leon          1
Henny         1
Aaron         1
Luca          1
Name: count, dtype: int64

In [96]:
test1 = ''
test2 = 'Chris'
test3 = spiele.loc[0, 'p6']
test4 = spiele.loc[0, 'p4']

if test3:
    print('das sollte nicht')
if test4:
    print('das sollte')

das sollte


In [64]:
for elem in spiele['p3']:
    if elem:
        print('Ja:', elem)
    else: print('Nö', elem)

Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Daisy
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Valdemar
Nö 
Ja: Valdemar
Ja: Aaron
Ja: Leon
Ja: Valdemar
Ja: Leon
Ja: Aaron
Ja: Leon
Ja: Leon
Ja: Daisy
Ja: Phia
Ja: Emilio
Ja: Phia
Ja: Leon
Ja: André
Ja: Ole
Ja: Phia
Ja: Ole
Ja: André
Ja: Phia
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Phia
Ja: Merlin
Ja: Phia
Ja: Marian
Ja: Daisy
Ja: Marian
Ja: Valdemar
Nö 
Nö 
Nö 
Nö 
Ja: Valdemar
Ja: Valdemar
Ja: Daisy
Ja: Marian
Ja: Valdemar
Ja: Chris
Ja: Daisy
Ja: Emilio
Ja: Cuba
Ja: Emilio
Nö 
Nö 
Nö 
Ja: Valdemar
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Daisy
Ja: Nora
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Marie
Nö 
Ja: Marie
Nö 
Nö 
Nö 
Ja: Marie
Nö 
Nö 
Ja: Marie
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Merlin
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Nö 
Ja: Valdemar
Ja: Marie
Ja: Henny
Ja: Marian
J

In [69]:
if spiele.loc[1, 'p5']:
    print('problem', spiele.loc[1, 'p5'])

problem Timo


In [37]:
andre_partner

{'Aaron': [0, 0, 0],
 'Alina': [0, 0, 0],
 'André': [16, 10, 0.625],
 'Basti': [0, 0, 0],
 'Chris': [5, 2, 0.4],
 'Cuba': [0, 0, 0],
 'Daisy': [16, 3, 0.1875],
 'Eggi': [0, 0, 0],
 'Emilio': [0, 0, 0],
 'Friedrich': [3, 1, 0.3333333333333333],
 'Gina': [0, 0, 0],
 'Hannes': [0, 0, 0],
 'Henny': [0, 0, 0],
 'Jonte': [0, 0, 0],
 'Konni': [0, 0, 0],
 'Konrad': [0, 0, 0],
 'Leon': [3, 3, 1.0],
 'Lin': [0, 0, 0],
 'Lorenz': [0, 0, 0],
 'Luca': [0, 0, 0],
 'Marian': [22, 15, 0.6818181818181818],
 'Marie': [6, 20, 3.3333333333333335],
 'Max': [0, 0, 0],
 'Merlin': [7, 9, 1.2857142857142858],
 'Michi': [0, 0, 0],
 'Moritz': [0, 0, 0],
 'Mümmel': [0, 0, 0],
 'Nikita': [0, 0, 0],
 'Nora': [6, 6, 1.0],
 'Ole': [4, 4, 1.0],
 'Olek': [3, 1, 0.3333333333333333],
 'Phia': [34, 23, 0.6764705882352942],
 'Philipp': [7, 4, 0.5714285714285714],
 'Richard': [0, 0, 0],
 'Tabea': [3, 3, 1.0],
 'Timo': [0, 0, 0],
 'Tobi': [0, 0, 0],
 'Valdemar': [15, 14, 0.9333333333333333],
 'Verena': [6, 2, 0.3333333333333

In [25]:
# winrate fenster
fenster = spiele[spiele['fenster'] == 1]
fenster
sum(fenster['fenster'])/len(fenster)
sum(fenster['WLt1'])/len(fenster)

0.5192307692307693

## Test Area

Maybe represent teams as dicts of player names?

In [27]:


dict_test = {player: [0, 0] for player in players}
#dict_test['Team1'] = [2, 1]
#dict_test['Team1'][1] += 1

dict_test['Chris'][1] += 1
dict_test

{'Aaron': [0, 0],
 'Alina': [0, 0],
 'André': [0, 0],
 'Basti': [0, 0],
 'Chris': [0, 1],
 'Cuba': [0, 0],
 'Daisy': [0, 0],
 'Eggi': [0, 0],
 'Emilio': [0, 0],
 'Friedrich': [0, 0],
 'Gina': [0, 0],
 'Hannes': [0, 0],
 'Henny': [0, 0],
 'Jonte': [0, 0],
 'Konni': [0, 0],
 'Konrad': [0, 0],
 'Leon': [0, 0],
 'Lin': [0, 0],
 'Lorenz': [0, 0],
 'Luca': [0, 0],
 'Marian': [0, 0],
 'Marie': [0, 0],
 'Max': [0, 0],
 'Merlin': [0, 0],
 'Michi': [0, 0],
 'Moritz': [0, 0],
 'Mümmel': [0, 0],
 'Nikita': [0, 0],
 'Nora': [0, 0],
 'Ole': [0, 0],
 'Olek': [0, 0],
 'Phia': [0, 0],
 'Philipp': [0, 0],
 'Richard': [0, 0],
 'Tabea': [0, 0],
 'Timo': [0, 0],
 'Tobi': [0, 0],
 'Valdemar': [0, 0],
 'Verena': [0, 0],
 'Vincent': [0, 0],
 'Zoë': [0, 0]}

In [40]:
spiele['p2'].value_counts()

p2
Emilio       39
Valdemar     32
André        24
Phia         22
Chris        22
Marian       20
Daisy        13
Philipp       9
Mümmel        8
Merlin        8
Friedrich     6
              4
Aaron         3
Lorenz        2
Verena        2
Nora          2
Luca          2
Basti         2
Max           2
Ole           2
Marie         2
Richard       1
Hannes        1
Vincent       1
Zoë           1
Gina          1
Moritz        1
Name: count, dtype: int64

In [84]:
test1 = ''
if test1:
    print('hallo')
test2 = "Chris"
if test2:
    print('hallo2')
test_dict = {}
if test1 and test1 in test_dict:
    print('sos')
else: print('sus')

hallo2
sus


In [90]:

#partner_test = pd.DataFrame(partner_andre, columns = ['Name', 'Total Games'])
#partner_test['Wins'] = partner_wins_andre
#partner_test.set_index('Name', inplace = True)

#partner_test.loc['Phia', 'Wins'] = partner_wins_andre['Phia']
#for key in partner_wins_andre:
#    partner_test.loc[key, 'Wins'] = partner_wins_andre[key]
#partner_test['Wins'].fillna(0, inplace = True)
#partner_test['Winrate'] = partner_test['Wins']/partner_test['Total Games']
#partner_test

In [89]:
andre, partner_andre = winrate('André') 
partner_andre.sort_values('Winrate', ascending = False)

Total games: 118
Winrate André: 53.39%



,Total Games,Wins,Winrate
Name,,,
Henny,3,8.0,2.666667
Marie,3,6.0,2.000000
Valdemar,16,21.0,1.312500
Luca,9,11.0,1.222222
Philipp,4,4.0,1.000000
Mümmel,5,5.0,1.000000
Marian,15,11.0,0.733333
Phia,24,17.0,0.708333
Emilio,16,10.0,0.625000


In [34]:
partner_wins_andre

{'Phia': 17,
 'Valdemar': 11,
 'Emilio': 7,
 'Mümmel': 1,
 'Michi': 1,
 'Philipp': 4,
 'Merlin': 1,
 'Marian': 7,
 'Chris': 11,
 'Basti': 1,
 'Aaron': 1,
 'Luca': 1}

In [26]:
winrate('Friedrich')

Total games: 18
Winrate Friedrich: 50.00%



In [27]:
partner_test

NameError: name 'partner_test' is not defined

In [9]:
andre, partner_andre = winrate('André') 
partner_andre

Total games: 118
Winrate André: 53.39%



[('Phia', 24),
 ('Chris', 20),
 ('Valdemar', 16),
 ('Emilio', 16),
 ('Marian', 15),
 ('Luca', 9),
 ('Daisy', 7),
 ('Merlin', 7),
 ('Aaron', 6),
 ('Mümmel', 5),
 ('Michi', 4),
 ('Philipp', 4),
 ('Basti', 3),
 ('Marie', 3),
 ('Henny', 3),
 ('Timo', 2),
 ('Gina', 2)]

In [67]:
# remove empty string
partner_andre = [x for x in partner_andre if x]
# remove the player themselves
partner_andre = [x for x in partner_andre if x!= 'André']
partner_andre
partners = Counter(partner_andre).most_common()
partners

[(('Phia', 24), 1),
 (('Chris', 20), 1),
 (('Valdemar', 16), 1),
 (('Emilio', 16), 1),
 (('Marian', 15), 1),
 (('Luca', 9), 1),
 (('Daisy', 7), 1),
 (('Merlin', 7), 1),
 (('Aaron', 6), 1),
 (('Mümmel', 5), 1),
 (('Michi', 4), 1),
 (('Philipp', 4), 1),
 (('Basti', 3), 1),
 (('Marie', 3), 1),
 (('Henny', 3), 1),
 (('Timo', 2), 1),
 (('Gina', 2), 1)]

In [65]:
partners[0]

('Phia', 24)

In [24]:
test = []
type(test)
test.append(['Ole', 'Marian'])
test

[['Ole', 'Marian']]

In [9]:
andre

,p1,p2,p3,p4,p5,p6,tname1,tname2,datum,WLt1,...,p2Becher,p3Becher,t1Becher,p4Becher,p5Becher,p6Becher,t2Becher,1v1,X-Runden Regel,Eigentore
0,Phia,Friedrich,,André,Timo,,Team Pferd *hüüh*,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
1,Emilio,Philipp,,André,Timo,,Saufomat,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
2,André,Phia,,Valdemar,Philipp,,Umnierter Blitzer,Team Phildemar aka Ja!,2022-05-14,0,...,,,,,,,,,,
3,Chris,Emilio,,André,Phia,,Klaus&Klaus,Umnierter Blitzer,2022-05-14,0,...,,,,,,,,,,
4,André,Valdemar,,Marian,Chris,,Rekkleslover69,Koks & Griesbrei,2022-01-12,1,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Daisy,Emilio,,Phia,André,,Graceful Geezers,Pieps ÖHH & KraKraKra,2024-05-26,0,...,2.0,,5.0,2.0,2.0,,4.0,,1.0,
114,Phia,André,,Marian,Chris,,Pieps ÖHH & KraKraKra,TT-Torsten und Thorben,2024-05-26,1,...,1.0,,9.0,3.0,4.0,,7.0,,,
115,Phia,André,,Philipp,Valdemar,,Pieps ÖHH & KraKraKra,Borosboiiis,2024-05-26,1,...,3.0,,6.0,2.0,2.0,,4.0,,,
116,Phia,Chris,,André,Marian,,DD-Detlef und Doris,Geckabor Gangster,2024-05-26,0,...,2.0,,3.0,3.0,3.0,,6.0,,,


In [23]:
andre

,p1,p2,p3,p4,p5,p6,tname1,tname2,datum,WLt1,...,p2Becher,p3Becher,t1Becher,p4Becher,p5Becher,p6Becher,t2Becher,1v1,X-Runden Regel,Eigentore
0,Phia,Friedrich,,André,Timo,,Team Pferd *hüüh*,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
1,Emilio,Philipp,,André,Timo,,Saufomat,Team Samenlando,2021-11-26,1,...,,,,,,,,,,
2,André,Phia,,Valdemar,Philipp,,Umnierter Blitzer,Team Phildemar aka Ja!,2022-05-14,0,...,,,,,,,,,,
3,Chris,Emilio,,André,Phia,,Klaus&Klaus,Umnierter Blitzer,2022-05-14,0,...,,,,,,,,,,
4,André,Valdemar,,Marian,Chris,,Rekkleslover69,Koks & Griesbrei,2022-01-12,1,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Daisy,Emilio,,Phia,André,,Graceful Geezers,Pieps ÖHH & KraKraKra,2024-05-26,0,...,2.0,,5.0,2.0,2.0,,4.0,,1.0,
114,Phia,André,,Marian,Chris,,Pieps ÖHH & KraKraKra,TT-Torsten und Thorben,2024-05-26,1,...,1.0,,9.0,3.0,4.0,,7.0,,,
115,Phia,André,,Philipp,Valdemar,,Pieps ÖHH & KraKraKra,Borosboiiis,2024-05-26,1,...,3.0,,6.0,2.0,2.0,,4.0,,,
116,Phia,Chris,,André,Marian,,DD-Detlef und Doris,Geckabor Gangster,2024-05-26,0,...,2.0,,3.0,3.0,3.0,,6.0,,,


halo2


In [18]:
p1 = andre['p1'].value_counts()
p1

p1
André        27
Phia         23
Emilio       12
Marian       11
Valdemar     10
Chris        10
Merlin        6
Luca          3
Daisy         3
Verena        2
Eggi          2
Gina          2
Olek          1
Michi         1
Jonte         1
Tobi          1
Friedrich     1
Aaron         1
Mümmel        1
Name: count, dtype: int64

In [41]:
for player in players:
    winrate(player)
    

Total games: 21
Winrate Aaron: 47.62%

Total games: 3
Winrate Alina: 66.67%

Total games: 118
Winrate André: 53.39%

Total games: 8
Winrate Basti: 25.00%

Total games: 87
Winrate Chris: 47.13%

Total games: 3
Winrate Cuba: 0.00%



KeyError: 'Wins'

In [18]:
# OLD WINRATE
def winrate2(player):
    df = spiele.copy(deep = True)
    # use this pattern to get rid of unwanted whitespaces and to make sure 'Ole' is not found when searching for 'Olek'
    pattern = rf'\b{player}\b'
    # important: reset the index because otherwise it will not start at 0 and the for loop wont work
    # drop = True: don't include the old indexes as a column
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    team = -1
    games = 0
    wins = 0
    partner = []
    player_number = 'p0'
    partner_games = {}
    partner_wins = {}
    partner1 = ''
    partner2 = ''
    for i in range(len(df)):
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p2'])
            partner.append(df.loc[i, 'p3'])
            #if df.loc[i, 'p2'] in partner_games
            #partner_games[df.loc[i, 'p2']] += 1
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p1'])
            partner.append(df.loc[i, 'p3'])   
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p1'])
            partner.append(df.loc[i, 'p2'])
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p5'])
            partner.append(df.loc[i, 'p6']) 
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p4'])
            partner.append(df.loc[i, 'p6']) 
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            # add both other players of team 1 as partners
            partner.append(df.loc[i, 'p4'])
            partner.append(df.loc[i, 'p5']) 
        else:
            print("Player {player} not found in row {row}".format(player = player, row = i))
        if team == 1:
            games += 1
            wins = wins + df.loc[i, 'WLt1']
        elif team == 2:
            games += 1
            wins = wins + df.loc[i, 'WLt2']
        else:
            print("Team not set correctly on Player {player} in row {row}".format(player = player, row = i))
    winrate = wins/games*100        
    print('Total games: {games}'.format(games=games))
    print('Winrate {player}: {winrate:.2f}%\n'.format(player=player, winrate=winrate))
    # remove empty string
    partner = [x for x in partner if x]
    # remove the player themselves, maybe not needed anymore
    #partner = [x for x in partner if x!= player]
    # count the occurences of partners and return them sorted by most frequent
    partner_count = Counter(partner).most_common()
    return df, partner_count

In [ ]:
# OLD WINRATE
def winrate3(player):
    df = spiele.copy(deep = True)
    # use this pattern to get rid of unwanted whitespaces and to make sure 'Ole' is not found when searching for 'Olek'
    pattern = rf'\b{player}\b'
    # important: reset the index because otherwise it will not start at 0 and the for loop wont work
    # drop = True: don't include the old indexes as a column
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    team = -1
    games = 0
    wins = 0
    partner = []
    player_number = 'p0'
    partner_games = {}
    partner_wins = {}
    partner1 = ''
    partner2 = ''
    for i in range(len(df)):
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p2']:
                partner1 = df.loc[i, 'p2']
                partner.append(partner1)
                #if partner1 in partner_games:
                #    partner_games[partner1] += 1
                #else: partner_games[partner1] = 1
            if partner2 != df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)  
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p2']:
                partner2 = df.loc[i, 'p2']
                partner.append(partner2)
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p5']:
                partner1 = df.loc[i, 'p5']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            # add both other players of team 1 as partners
            if partner1 != df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
            if partner2 != df.loc[i, 'p5']:
                partner2 = df.loc[i, 'p5']
                partner.append(partner2)
        else:
            print("Player {player} not found in row {row}".format(player = player, row = i))
        if team == 1:
            games += 1
            current_win = df.loc[i, 'WLt1']
            if current_win:
                wins = wins + 1
                if partner1 in partner_wins:
                    partner_wins[partner1] += 1
                else: partner_wins[partner1] = 1
                if partner2 in partner_games:
                    partner_games[partner2] += 1
                else: partner_games[partner2] = 1
        elif team == 2:
            games += 1
            current_win = df.loc[i, 'WLt2']
            if current_win:
                wins = wins + 1
                if partner1 in partner_wins:
                    partner_wins[partner1] += 1
                else: partner_wins[partner1] = 1
                if partner2 in partner_games:
                    partner_games[partner2] += 1
                else: partner_games[partner2] = 1
        else:
            print("Team not set correctly on Player {player} in row {row}".format(player = player, row = i))
    winrate = wins/games*100        
    print('Total games: {games}'.format(games=games))
    print('Winrate {player}: {winrate:.2f}%\n'.format(player=player, winrate=winrate))
    # remove empty string
    partner = [x for x in partner if x]
    # remove the player themselves, maybe not needed anymore
    #partner = [x for x in partner if x!= player]
    # count the occurences of partners and return them sorted by most frequent
    partner_count = Counter(partner).most_common()
    return df, partner_count, partner_wins

In [ ]:
def winrate4(player):
    df = spiele.copy(deep = True)
    # use this pattern to get rid of unwanted whitespaces and to make sure 'Ole' is not found when searching for 'Olek'
    pattern = rf'\b{player}\b'
    # important: reset the index because otherwise it will not start at 0 and the for loop wont work
    # drop = True: don't include the old indexes as a column
    df = df[df[player_cols].apply(lambda row: row.str.contains(pattern).any(), axis=1)].reset_index(drop=True)
    team = -1
    games = 0
    wins = 0
    partner = []
    partner_dict = {player: [0, 0] for player in players}
    player_number = 'p0'
#    partner_games = {}
    partner_wins = {}
    partner1 = ''
    partner2 = ''
    for i in range(len(df)):
        if df.loc[i, 'p1'] == player:
            player_number = 'p1'
            team = 1
            # add both other players of team 1 as partners
            if df.loc[i, 'p2']:
                partner1 = df.loc[i, 'p2']
                partner.append(partner1)
                partner_dict[partner1][0] += 1 
                #if partner1 in partner_games:
                #    partner_games[partner1] += 1
                #else: partner_games[partner1] = 1
            if df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)
                partner_dict[partner2][0] += 1 
        elif df.loc[i, 'p2'] == player:
            player_number = 'p2'
            team = 1
            # add both other players of team 1 as partners
            if df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
                partner_dict[partner1][0] += 1 
            if df.loc[i, 'p3']:
                partner2 = df.loc[i, 'p3']
                partner.append(partner2)
                partner_dict[partner2][0] += 1  
        elif df.loc[i, 'p3'] == player:
            player_number = 'p3'
            team = 1
            # add both other players of team 1 as partners
            if df.loc[i, 'p1']:
                partner1 = df.loc[i, 'p1']
                partner.append(partner1)
                partner_dict[partner1][0] += 1 
            if df.loc[i, 'p2']:
                partner2 = df.loc[i, 'p2']
                partner.append(partner2)
                partner_dict[partner2][0] += 1 
        elif df.loc[i, 'p4'] == player:
            player_number = 'p4'
            team = 2
            # add both other players of team 1 as partners
            if df.loc[i, 'p5']:
                partner1 = df.loc[i, 'p5']
                partner.append(partner1)
                partner_dict[partner1][0] += 1 
            if df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
                partner_dict[partner2][0] += 1 
        elif df.loc[i, 'p5'] == player:
            player_number = 'p5'
            team = 2
            # add both other players of team 1 as partners
            if df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
                partner_dict[partner1][0] += 1 
            if df.loc[i, 'p6']:
                partner2 = df.loc[i, 'p6']
                partner.append(partner2)
                partner_dict[partner2][0] += 1 
        elif df.loc[i, 'p6'] == player:
            player_number = 'p6'
            team = 2
            # add both other players of team 1 as partners
            if df.loc[i, 'p4']:
                partner1 = df.loc[i, 'p4']
                partner.append(partner1)
                partner_dict[partner1][0] += 1 
            if df.loc[i, 'p5']:
                partner2 = df.loc[i, 'p5']
                partner.append(partner2)
                partner_dict[partner2][0] += 1 
        else:
            print("Player {player} not found in row {row}".format(player = player, row = i))
        if team == 1:
            games += 1
            current_win = df.loc[i, 'WLt1']
            if current_win:
                wins = wins + 1
                if partner1:
                    partner_dict[partner1][1] += 1 
                    if partner1 in partner_wins:
                        partner_wins[partner1] += 1
                    else: partner_wins[partner1] = 1
                if partner2:
                    partner_dict[partner2][1] += 1 
                    if partner2 in partner_wins:
                        partner_wins[partner2] += 1
                    else: partner_wins[partner2] = 1
        elif team == 2:
            games += 1
            current_win = df.loc[i, 'WLt2']
            if current_win:
                wins = wins + 1
                if partner1:  
                    partner_dict[partner1][1] += 1 
                    if partner1 in partner_wins:
                        partner_wins[partner1] += 1
                    else: partner_wins[partner1] = 1
                if partner2:
                    partner_dict[partner2][1] += 1 
                    if partner2 in partner_wins:
                        partner_wins[partner2] += 1
                    else: partner_wins[partner2] = 1
        else:
            print("Team not set correctly on Player {player} in row {row}".format(player = player, row = i))
    winrate = wins/games*100        
    print('Total games: {games}'.format(games=games))
    print('Winrate {player}: {winrate:.2f}%\n'.format(player=player, winrate=winrate))
    # remove empty string
    partner = [x for x in partner if x]
    # remove the player themselves, maybe not needed anymore
    #partner = [x for x in partner if x!= player]
    # count the occurences of partners and return them sorted by most frequent
    partner_count = Counter(partner).most_common()
    # pack the partner names & number of total games into a df 
    partner_df = pd.DataFrame(partner_count, columns = ['Name', 'Total Games'])
    # make the names the index
    partner_df.set_index('Name', inplace = True)
    # add the parter wins
    for key in partner_wins:
        partner_df.loc[key, 'Wins'] = partner_wins[key]
    partner_df['Wins'].fillna(0, inplace = True)
    partner_df['Winrate'] = partner_df['Wins']/partner_df['Total Games']
    # add winrate to partner_dict
    for elem in partner_dict:
        games_tmp = partner_dict[elem][0]
        wins_tmp = partner_dict[elem][1]
        # handle division by 0
        if games_tmp !=0:
            winrate_tmp = wins_tmp / games_tmp
        else: 
            winrate_tmp = 0
        partner_dict[elem].append(winrate_tmp)
    return partner_df, partner_dict